##### TV Script Generation

In this project, you'll generate your own Simpsons (https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs. You'll be using part of the Simpsons dataset (https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons. The Neural Network you'll build will generate a new TV script for a scene at Moe's Tavern (https://simpsonswiki.com/wiki/Moe's_Tavern).

##### Get the Data

The data is already provided for you. You'll be using a subset of the original dataset. It consists of only the scenes in Moe's Tavern. This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

#### The following are some helper functions students can use in their code

In [11]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import os
import pickle


def load_data(path):
    
    #input_file = os.path.join(path)
    with open(path, mode='r') as f:
        data = f.read()

    return data


def preprocess_and_save_data(dataset_path, token_lookup, create_lookup_tables):
    
    text = load_data(dataset_path)
    
    text = text[81:]

    token_dict = token_lookup()
    for key, token in token_dict.items():
        text = text.replace(key, ' {} '.format(token))

    text = text.lower()
    text = text.split()

    vocab_to_int, int_to_vocab = create_lookup_tables(text)
    int_text = [vocab_to_int[word] for word in text]
    pickle.dump((int_text, vocab_to_int, int_to_vocab, token_dict), open('preprocess.p', 'wb'))


def load_preprocess():
    
    return pickle.load(open('preprocess.p', mode='rb'))


def save_params(params):
    
    pickle.dump(params, open('params.p', 'wb'))


def load_params():
    
    return pickle.load(open('params.p', mode='rb'))
  

data_dir = '/gdrive/My Drive/Colab Notebooks/moes_tavern_lines.txt'
text = load_data(data_dir)

text = text[81:]

##### Explore the Data

Play around with view_sentence_range to view different parts of the data.


In [15]:
view_sentence_range = (0, 10)


import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




#### Implement Preprocessing Functions

The first thing to do to any dataset is preprocessing. Implement the following preprocessing functions below:
  

*   Lookup Table
*   Tokenize Punctuation
   

##### Lookup Table

To create a word embedding, you first need to transform the words to ids. In this function, create two dictionaries:   
    

*  Dictionary to go from the words to an id, we'll call vocab_to_int
*  Dictionary to go from the id to word, we'll call int_to_vocab

Return these dictionaries in the following tuple (vocab_to_int, int_to_vocab)


In [0]:
import numpy as np

def create_lookup_tables(text):
    
    vocab_to_int = dict((word, index) for index, word in enumerate(set(text)))
    int_to_vocab = dict((index, word) for index, word in enumerate(set(text)))
    
    return (vocab_to_int, int_to_vocab)


#### Tokenize Punctuation

We'll be splitting the script into a word array using spaces as delimiters. However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function token_lookup to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". Create a dictionary for the following symbols where the symbol is the key and value is the token:
   

*  Period ( . )
*  Comma ( , )
*  Quotation Mark ( " )
*  Semicolon ( ; )
*  Exclamation mark ( ! )
*  Question mark ( ? )
*  Left Parentheses ( ( )
*  Right Parentheses ( ) )
*   Dash ( -- )
*  Return ( \n )
    

This dictionary will be used to token the symbols and add the delimiter (space) around it. This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".


In [0]:
def token_lookup():
    
    values = ['Period','Comma','Quotation_Mark','Semicolon','Exclamation_mark','Question_mark',
              'Left_Parentheses','Right_Parentheses','Dash','Return']
    keys = ['.', ',', '"', ';', '!', '?', '(', ')', '--','\n']
    
    return (dict(zip(keys,values)))


#### Preprocess all the data and save it

Running the code cell below will preprocess all the data and save it to file.


In [0]:
preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

#### Check Point

This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.


In [0]:
import numpy as np

int_text, vocab_to_int, int_to_vocab, token_dict = load_preprocess()

#### Build the Neural Network

You'll build the components necessary to build a RNN by implementing the following functions below:

    get_inputs
    get_init_cell
    get_embed
    build_rnn
    build_nn
    get_batches

#### Check the Version of TensorFlow and Access to GPU

In [20]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.14.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: No GPU found. Please use a GPU to train your neural network.
  # This is added back by InteractiveShellApp.init_path()


#### Input

Implement the get_inputs() function to create TF Placeholders for the Neural Network. It should create the following placeholders:

    

*   Input text placeholder named "input" using the TF Placeholder name parameter.
*   Targets placeholder
*   Learning Rate placeholder   
    

Return the placeholders in the following tuple (Input, Targets, LearningRate)


In [0]:
def get_inputs():
    
    input = tf.placeholder(tf.int32, [None, None], name= 'input')
    targets = tf.placeholder(tf.int32, [None, None], name= 'targets')
    learning_rate = tf.placeholder(tf.float32, name= 'learning_rate')
    
    return input, targets, learning_rate



#### Build RNN Cell and Initialize

Stack one or more BasicLSTMCells in a MultiRNNCell.


*   The Rnn size should be set using rnn_size
*    Initalize Cell State using the MultiRNNCell's zero_state() function
  *   Apply the name "initial_state" to the initial state using tf.identity()  
        

Return the cell and initial state in the following tuple (Cell, InitialState)


In [0]:
def get_init_cell(batch_size, rnn_size):
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name= 'initial_state')
    
    return (cell, initial_state)


#### Word Embedding

Apply embedding to input_data using TensorFlow. Return the embedded sequence.


In [0]:
def get_embed(input_data, vocab_size, embed_dim):
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim),-1,1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


#### Build RNN

You created a RNN Cell in the get_init_cell() function. Time to use the cell to create a RNN.
   

*   Build the RNN using the tf.nn.dynamic_rnn()
  *   Apply the name "final_state" to the final state using tf.identity()
      

Return the outputs and final_state state in the following tuple (Outputs, FinalState)


In [0]:
def build_rnn(cell, inputs):
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name= 'final_state')
    
    return (outputs, final_state)



#### Build the Neural Network

Apply the functions you implemented above to:
    

*   Apply embedding to input_data using your get_embed(input_data, vocab_size, embed_dim) function.
*  Build RNN using cell and your build_rnn(cell, inputs) function.
*   Apply a fully connected layer with a linear activation and vocab_size as the number of outputs.
   

Return the logits and final state in the following tuple (Logits, FinalState)


In [0]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    
    embedded = get_embed(input_data, vocab_size, rnn_size)
    outputs, final_state = build_rnn(cell, embedded)
    output = tf.reshape(outputs, [-1, rnn_size])
    softmax_w = tf.Variable(tf.truncated_normal((rnn_size, vocab_size), stddev=0.01))
    softmax_b = tf.Variable(tf.zeros(vocab_size))
    logits = tf.add(tf.matmul(output, softmax_w), softmax_b)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state



In [0]:
def get_batches(int_text, batch_size, seq_length):
    
    n_batches = int(len(int_text) / (batch_size * seq_length))
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])
    ydata[-1] = xdata[0]
    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    return np.asarray(list(zip(x_batches, y_batches)))



#### Neural Network Training Hyperparameters

Tune the following parameters:    

*   Set num_epochs to the number of epochs.
*   Set batch_size to the batch size.
*   Set rnn_size to the size of the RNNs.
*   Set embed_dim to the size of the embedding.
*   Set seq_length to the length of sequence.
*   Set learning_rate to the learning rate.
*   Set show_every_n_batches to the number of batches the neural network should print progress.

In [0]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 32
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 20


save_dir = './save'

#### Build the Graph

Build the graph using the neural network you implemented.


In [28]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

W0803 02:43:35.436472 140491114170240 deprecation.py:323] From <ipython-input-22-5f316f56cdaa>:3: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0803 02:43:35.438598 140491114170240 deprecation.py:323] From <ipython-input-22-5f316f56cdaa>:4: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0803 02:43:35.479353 140491114170240 deprecation.py:323] From <ipython-input-24-c69f37a07daa>:3: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W080

In [29]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/16   train_loss = 8.822
Epoch   1 Batch    4/16   train_loss = 5.805
Epoch   2 Batch    8/16   train_loss = 5.323
Epoch   3 Batch   12/16   train_loss = 4.811
Epoch   5 Batch    0/16   train_loss = 4.493
Epoch   6 Batch    4/16   train_loss = 4.293
Epoch   7 Batch    8/16   train_loss = 4.158
Epoch   8 Batch   12/16   train_loss = 3.896
Epoch  10 Batch    0/16   train_loss = 3.696
Epoch  11 Batch    4/16   train_loss = 3.563
Epoch  12 Batch    8/16   train_loss = 3.488
Epoch  13 Batch   12/16   train_loss = 3.320
Epoch  15 Batch    0/16   train_loss = 3.164
Epoch  16 Batch    4/16   train_loss = 3.087
Epoch  17 Batch    8/16   train_loss = 3.050
Epoch  18 Batch   12/16   train_loss = 2.963
Epoch  20 Batch    0/16   train_loss = 2.847
Epoch  21 Batch    4/16   train_loss = 2.754
Epoch  22 Batch    8/16   train_loss = 2.728
Epoch  23 Batch   12/16   train_loss = 2.686
Epoch  25 Batch    0/16   train_loss = 2.581
Epoch  26 Batch    4/16   train_loss = 2.527
Epoch  27 

#### Save Parameters

Save seq_length and save_dir for generating a new TV script.


In [0]:
save_params((seq_length, save_dir))

### Checkpoint

In [0]:
import tensorflow as tf
import numpy as np


_, vocab_to_int, int_to_vocab, token_dict = load_preprocess()
seq_length, load_dir = load_params()

### Implement Generate Functions
#### Get Tensors

Get tensors from loaded_graph using the function get_tensor_by_name(). Get the tensors using the following names:

    "input:0"
    "initial_state:0"
    "final_state:0"
    "probs:0"

Return the tensors in the following tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

In [0]:
def get_tensors(loaded_graph):
    
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)



#### Choose Word

Implement the pick_word() function to select the next word using probabilities.


In [0]:
def pick_word(probabilities, int_to_vocab):
    
    x = np.argmax(probabilities)
    return int_to_vocab[x]

### Generate TV Script

This will generate the TV script for you. Set gen_length to the length of TV script you want to generate.


In [60]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'


loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0,dyn_seq_length-1,:], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(into phone) moe's tavern...
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.
moe_szyslak:(to homer) i don't know, then i popped out backwards and on fire.

